In [1]:
import pandas as pd
import numpy as np
PATH = '/Users/orendar/Downloads/dataverse_files/'

In [3]:
train_df = pd.read_csv(f'{PATH}train.csv', index_col = 0)
train_df.head()

,year,camp.length,deminc,base.poll,base.und,office,d.gone.neg,d.gone.neg.l1,d.gone.neg.l2,d.neg.frac.l3,...,y2002,y2000,pos.prob.subset,missing,bart.sw,gbm.sw,gam.sw,bart.pscore,gbm.pscore,gam.pscore
1,2002,23,1,45.744681,6.0,0,0,0,0,0.000000,...,1,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,23,1,45.744681,6.0,0,0,0,0,0.000000,...,1,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,23,1,45.744681,6.0,0,1,1,0,0.000000,...,1,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,23,1,45.744681,6.0,0,1,1,1,0.000000,...,1,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN
5,2002,23,1,45.744681,6.0,0,1,1,1,0.071429,...,1,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
test_df = pd.read_csv(f'{PATH}test.csv',  index_col = 0)
test_df.head()

,year,camp.length,deminc,base.poll,base.und,office,d.gone.neg,d.gone.neg.l1,d.gone.neg.l2,d.neg.frac.l3,...,missing,bart.sw,gbm.sw,gam.sw,bart.pscore,gbm.pscore,gam.pscore,gam.pred,gbm.pred,bart.pred
1,2002,13,0,44.736842,24.0,1,1,1,1,1.000000,...,False,NaN,NaN,NaN,NaN,NaN,NaN,0.853041,0.925549,0.976336
2,2006,23,1,63.636364,12.0,0,0,0,0,0.000000,...,False,NaN,NaN,NaN,NaN,NaN,NaN,0.013914,0.093114,0.066920
3,2002,23,0,62.765957,6.0,0,1,1,1,0.176471,...,False,NaN,NaN,NaN,NaN,NaN,NaN,0.921248,0.908793,0.895841
4,2006,27,1,62.637363,9.0,1,0,0,0,0.090909,...,False,NaN,NaN,NaN,NaN,NaN,NaN,0.222096,0.134856,0.064143
5,2006,15,1,66.455237,9.0,0,0,0,0,0.000000,...,False,NaN,NaN,NaN,NaN,NaN,NaN,0.044348,0.219977,0.216478


In [4]:
features = ["d.gone.neg.l1", "d.gone.neg.l2", "d.neg.frac.l3", "week",
            "camp.length", "base.poll", "y2000", "y2002", "y2004", "y2006",
            "base.und", "office", "dem.polls.l1", "dem.polls.l2", "neg.rep.l1",
            "neg.rep.l2", "r.neg.frac.l2", "r.neg.frac.l3", "num.rep.l1",
            "num.rep.l2", "num.dem.l1", "num.dem.l2", "undother.l1",
            "undother.l2", "deminc", "dem.contrib.l1", "dem.contrib.l2",
            "rep.contrib.l1", "rep.contrib.l2"]

target = "d.gone.neg"

train_df = train_df[features+[target]].dropna()
y_train = train_df[target]
x_train = train_df.drop(target, axis=1)
test_df = test_df[features+[target]].dropna()
y_test = test_df[target]
x_test = test_df.drop(target, axis=1)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((818, 29), (818,), (274, 29), (274,))

In [5]:
from sklearn.ensemble import *
from sklearn.metrics import *
rs = 42

In [6]:
gbc = GradientBoostingClassifier(n_estimators=100, max_features='sqrt', random_state=rs)
rfc = RandomForestClassifier(n_estimators=500, random_state=rs, n_jobs=-1)
etc = ExtraTreesClassifier(n_estimators=500, random_state=rs, n_jobs=-1)

In [7]:
gbc.fit(x_train, y_train)
rfc.fit(x_train, y_train)
etc.fit(x_train, y_train);

In [8]:
gbc_preds = gbc.predict(x_test)
rfc_preds = rfc.predict(x_test)
etc_preds = etc.predict(x_test)
gbc_prob_preds = gbc.predict_proba(x_test)
rfc_prob_preds = rfc.predict_proba(x_test)
etc_prob_preds = etc.predict_proba(x_test)
gbc_pos = gbc_prob_preds.T[1]
rfc_pos = rfc_prob_preds.T[1]
etc_pos = etc_prob_preds.T[1]

In [9]:
def rmse(preds): return mean_squared_error(y_test, preds)**.5
def auroc(preds): return roc_auc_score(y_test, preds)
def accuracy(preds): return accuracy_score(y_test, preds)
def logloss(preds): return log_loss(y_test, preds)

In [10]:
rmse(gbc_pos), rmse(rfc_pos), rmse(etc_pos)

(0.2911546724006993, 0.2997949664317435, 0.3041994879468953)

In [11]:
accuracy(gbc_preds), accuracy(rfc_preds), accuracy(etc_preds)

(0.8795620437956204, 0.8686131386861314, 0.8576642335766423)

In [12]:
auroc(gbc_pos), auroc(rfc_pos), auroc(etc_pos)

(0.9397605083088955, 0.933284457478006, 0.9353005865102639)

In [13]:
logloss(gbc_pos), logloss(rfc_pos), logloss(etc_pos)

(0.2821562723408976, 0.3029121745427702, 0.2943411109232128)

In [14]:
df = pd.DataFrame(data=[[accuracy(gbc_preds), rmse(gbc_pos), auroc(gbc_pos), logloss(gbc_pos)],
                        [accuracy(rfc_preds), rmse(rfc_pos), auroc(rfc_pos), logloss(rfc_pos)],
                        [accuracy(etc_preds), rmse(etc_pos), auroc(etc_pos), logloss(etc_pos)]],
                  index=['GBM', 'RF', 'ET'],
                  columns=['Accuracy', 'RMSE', 'AUROC', 'LogLoss'])
df.head()

,Accuracy,RMSE,AUROC,LogLoss
GBM,0.879562,0.291155,0.939761,0.282156
RF,0.868613,0.299795,0.933284,0.302912
ET,0.857664,0.304199,0.935301,0.294341


In [15]:
df.to_latex()

'\\begin{tabular}{lrrrr}\n\\toprule\n{} &  Accuracy &      RMSE &     AUROC &   LogLoss \\\\\n\\midrule\nGBM &  0.879562 &  0.291155 &  0.939761 &  0.282156 \\\\\nRF  &  0.868613 &  0.299795 &  0.933284 &  0.302912 \\\\\nET  &  0.857664 &  0.304199 &  0.935301 &  0.294341 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [16]:
from keras.models import *
from keras.layers import *
from keras.utils import *
from keras.optimizers import *
from sklearn.preprocessing import *
from clr import *
np.random.seed(42)

/Users/orendar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
x_train_mat, y_train_mat = x_train.as_matrix(), y_train.as_matrix().reshape(-1, 1)
x_test_mat, y_test_mat = x_test.as_matrix(), y_test.as_matrix().reshape(-1, 1)

In [18]:
scaler = StandardScaler()
x_train_norm = scaler.fit_transform(x_train_mat)
x_test_norm = scaler.transform(x_test_mat)

In [27]:
model = Sequential()
model.add(Dense(4096, input_dim=29, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile model
opt = optimizers.adam(amsgrad=True)
#opt = optimizers.SGD(momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['mse', 'accuracy'])

#callbacks arsenal
def sched(idx, lr): return lr*0.9 if idx % 10 == 0 else lr
lrsched = LearningRateScheduler(sched, verbose=0)
reducelr = ReduceLROnPlateau(factor=0.8, patience=2)
stop = EarlyStopping(patience=50)
clr = CyclicLR(base_lr=1e-5, max_lr=1e-4, step_size=15, mode='triangular', scale_mode='cycle')

In [28]:
# Fit the model
model.fit(x_train_norm, y_train, validation_data=(x_test_norm, y_test), epochs=3000, batch_size=512,
          callbacks=[clr, stop], verbose=2)

# evaluate the model
scores = model.evaluate(x_test_norm, y_test)
model.metrics_names, scores

Train on 818 samples, validate on 274 samples
Epoch 1/3000
 - 0s - loss: 0.7078 - mean_squared_error: 0.2573 - acc: 0.4120 - val_loss: 0.6985 - val_mean_squared_error: 0.2527 - val_acc: 0.4489
Epoch 2/3000
 - 0s - loss: 0.7007 - mean_squared_error: 0.2538 - acc: 0.4450 - val_loss: 0.6902 - val_mean_squared_error: 0.2485 - val_acc: 0.5182
Epoch 3/3000
 - 0s - loss: 0.6939 - mean_squared_error: 0.2504 - acc: 0.5147 - val_loss: 0.6783 - val_mean_squared_error: 0.2426 - val_acc: 0.5876
Epoch 4/3000
 - 0s - loss: 0.6800 - mean_squared_error: 0.2435 - acc: 0.6149 - val_loss: 0.6632 - val_mean_squared_error: 0.2351 - val_acc: 0.7044
Epoch 5/3000
 - 0s - loss: 0.6634 - mean_squared_error: 0.2353 - acc: 0.6797 - val_loss: 0.6456 - val_mean_squared_error: 0.2264 - val_acc: 0.7664
Epoch 6/3000
 - 0s - loss: 0.6424 - mean_squared_error: 0.2250 - acc: 0.7213 - val_loss: 0.6258 - val_mean_squared_error: 0.2168 - val_acc: 0.7847
Epoch 7/3000
 - 0s - loss: 0.6252 - mean_squared_error: 0.2166 - acc: 0.

Epoch 57/3000
 - 0s - loss: 0.3914 - mean_squared_error: 0.1189 - acc: 0.8533 - val_loss: 0.3834 - val_mean_squared_error: 0.1161 - val_acc: 0.8467
Epoch 58/3000
 - 0s - loss: 0.3925 - mean_squared_error: 0.1193 - acc: 0.8521 - val_loss: 0.3822 - val_mean_squared_error: 0.1157 - val_acc: 0.8467
Epoch 59/3000
 - 0s - loss: 0.3876 - mean_squared_error: 0.1179 - acc: 0.8521 - val_loss: 0.3814 - val_mean_squared_error: 0.1154 - val_acc: 0.8467
Epoch 60/3000
 - 0s - loss: 0.3893 - mean_squared_error: 0.1184 - acc: 0.8533 - val_loss: 0.3809 - val_mean_squared_error: 0.1153 - val_acc: 0.8467
Epoch 61/3000
 - 0s - loss: 0.3884 - mean_squared_error: 0.1179 - acc: 0.8557 - val_loss: 0.3806 - val_mean_squared_error: 0.1152 - val_acc: 0.8467
Epoch 62/3000
 - 0s - loss: 0.3884 - mean_squared_error: 0.1182 - acc: 0.8484 - val_loss: 0.3800 - val_mean_squared_error: 0.1149 - val_acc: 0.8467
Epoch 63/3000
 - 0s - loss: 0.3881 - mean_squared_error: 0.1179 - acc: 0.8484 - val_loss: 0.3790 - val_mean_squa

Epoch 113/3000
 - 0s - loss: 0.3306 - mean_squared_error: 0.0985 - acc: 0.8765 - val_loss: 0.3308 - val_mean_squared_error: 0.0992 - val_acc: 0.8577
Epoch 114/3000
 - 0s - loss: 0.3277 - mean_squared_error: 0.0971 - acc: 0.8790 - val_loss: 0.3298 - val_mean_squared_error: 0.0989 - val_acc: 0.8577
Epoch 115/3000
 - 0s - loss: 0.3263 - mean_squared_error: 0.0974 - acc: 0.8753 - val_loss: 0.3290 - val_mean_squared_error: 0.0987 - val_acc: 0.8577
Epoch 116/3000
 - 0s - loss: 0.3296 - mean_squared_error: 0.0983 - acc: 0.8741 - val_loss: 0.3283 - val_mean_squared_error: 0.0985 - val_acc: 0.8577
Epoch 117/3000
 - 0s - loss: 0.3267 - mean_squared_error: 0.0975 - acc: 0.8802 - val_loss: 0.3277 - val_mean_squared_error: 0.0984 - val_acc: 0.8577
Epoch 118/3000
 - 0s - loss: 0.3238 - mean_squared_error: 0.0963 - acc: 0.8778 - val_loss: 0.3273 - val_mean_squared_error: 0.0983 - val_acc: 0.8577
Epoch 119/3000
 - 0s - loss: 0.3245 - mean_squared_error: 0.0966 - acc: 0.8729 - val_loss: 0.3270 - val_me

Epoch 168/3000
 - 0s - loss: 0.2965 - mean_squared_error: 0.0877 - acc: 0.8839 - val_loss: 0.3084 - val_mean_squared_error: 0.0938 - val_acc: 0.8686
Epoch 169/3000
 - 0s - loss: 0.2964 - mean_squared_error: 0.0876 - acc: 0.8851 - val_loss: 0.3082 - val_mean_squared_error: 0.0937 - val_acc: 0.8686
Epoch 170/3000
 - 0s - loss: 0.2957 - mean_squared_error: 0.0874 - acc: 0.8826 - val_loss: 0.3079 - val_mean_squared_error: 0.0937 - val_acc: 0.8723
Epoch 171/3000
 - 0s - loss: 0.2924 - mean_squared_error: 0.0863 - acc: 0.8839 - val_loss: 0.3075 - val_mean_squared_error: 0.0936 - val_acc: 0.8723
Epoch 172/3000
 - 0s - loss: 0.2955 - mean_squared_error: 0.0873 - acc: 0.8826 - val_loss: 0.3071 - val_mean_squared_error: 0.0935 - val_acc: 0.8723
Epoch 173/3000
 - 0s - loss: 0.2907 - mean_squared_error: 0.0858 - acc: 0.8888 - val_loss: 0.3067 - val_mean_squared_error: 0.0934 - val_acc: 0.8723
Epoch 174/3000
 - 0s - loss: 0.2899 - mean_squared_error: 0.0855 - acc: 0.8912 - val_loss: 0.3063 - val_me

Epoch 223/3000
 - 0s - loss: 0.2727 - mean_squared_error: 0.0804 - acc: 0.8863 - val_loss: 0.2978 - val_mean_squared_error: 0.0919 - val_acc: 0.8723
Epoch 224/3000
 - 0s - loss: 0.2708 - mean_squared_error: 0.0798 - acc: 0.8888 - val_loss: 0.2977 - val_mean_squared_error: 0.0919 - val_acc: 0.8723
Epoch 225/3000
 - 0s - loss: 0.2715 - mean_squared_error: 0.0800 - acc: 0.8888 - val_loss: 0.2977 - val_mean_squared_error: 0.0919 - val_acc: 0.8723
Epoch 226/3000
 - 0s - loss: 0.2712 - mean_squared_error: 0.0801 - acc: 0.8912 - val_loss: 0.2976 - val_mean_squared_error: 0.0919 - val_acc: 0.8723
Epoch 227/3000
 - 0s - loss: 0.2722 - mean_squared_error: 0.0803 - acc: 0.8924 - val_loss: 0.2976 - val_mean_squared_error: 0.0919 - val_acc: 0.8723
Epoch 228/3000
 - 0s - loss: 0.2709 - mean_squared_error: 0.0798 - acc: 0.8900 - val_loss: 0.2975 - val_mean_squared_error: 0.0919 - val_acc: 0.8759
Epoch 229/3000
 - 0s - loss: 0.2684 - mean_squared_error: 0.0789 - acc: 0.8924 - val_loss: 0.2974 - val_me

Epoch 278/3000
 - 0s - loss: 0.2533 - mean_squared_error: 0.0738 - acc: 0.9034 - val_loss: 0.2937 - val_mean_squared_error: 0.0916 - val_acc: 0.8723
Epoch 279/3000
 - 0s - loss: 0.2539 - mean_squared_error: 0.0746 - acc: 0.8961 - val_loss: 0.2936 - val_mean_squared_error: 0.0916 - val_acc: 0.8723
Epoch 280/3000
 - 0s - loss: 0.2573 - mean_squared_error: 0.0759 - acc: 0.8936 - val_loss: 0.2935 - val_mean_squared_error: 0.0916 - val_acc: 0.8723
Epoch 281/3000
 - 0s - loss: 0.2539 - mean_squared_error: 0.0749 - acc: 0.8998 - val_loss: 0.2935 - val_mean_squared_error: 0.0916 - val_acc: 0.8723
Epoch 282/3000
 - 0s - loss: 0.2523 - mean_squared_error: 0.0739 - acc: 0.9022 - val_loss: 0.2934 - val_mean_squared_error: 0.0916 - val_acc: 0.8723
Epoch 283/3000
 - 0s - loss: 0.2515 - mean_squared_error: 0.0739 - acc: 0.8985 - val_loss: 0.2934 - val_mean_squared_error: 0.0916 - val_acc: 0.8723
Epoch 284/3000
 - 0s - loss: 0.2539 - mean_squared_error: 0.0747 - acc: 0.8998 - val_loss: 0.2934 - val_me

Epoch 333/3000
 - 0s - loss: 0.2386 - mean_squared_error: 0.0700 - acc: 0.9071 - val_loss: 0.2923 - val_mean_squared_error: 0.0917 - val_acc: 0.8686
Epoch 334/3000
 - 0s - loss: 0.2399 - mean_squared_error: 0.0705 - acc: 0.9034 - val_loss: 0.2923 - val_mean_squared_error: 0.0917 - val_acc: 0.8686
Epoch 335/3000
 - 0s - loss: 0.2381 - mean_squared_error: 0.0696 - acc: 0.9034 - val_loss: 0.2923 - val_mean_squared_error: 0.0917 - val_acc: 0.8686
Epoch 336/3000
 - 0s - loss: 0.2426 - mean_squared_error: 0.0711 - acc: 0.9059 - val_loss: 0.2923 - val_mean_squared_error: 0.0917 - val_acc: 0.8686
Epoch 337/3000
 - 0s - loss: 0.2408 - mean_squared_error: 0.0706 - acc: 0.9034 - val_loss: 0.2924 - val_mean_squared_error: 0.0917 - val_acc: 0.8686
Epoch 338/3000
 - 0s - loss: 0.2400 - mean_squared_error: 0.0705 - acc: 0.9022 - val_loss: 0.2924 - val_mean_squared_error: 0.0918 - val_acc: 0.8686
Epoch 339/3000
 - 0s - loss: 0.2390 - mean_squared_error: 0.0699 - acc: 0.9059 - val_loss: 0.2925 - val_me

(['loss', 'mean_squared_error', 'acc'],
 [0.29186007423992577, 0.09165919387210024, 0.8649635040847055])

In [29]:
nn_preds = model.predict_classes(x_test_norm)
nn_pos = model.predict_proba(x_test_norm)

In [30]:
accuracy(nn_preds), rmse(nn_pos), auroc(nn_pos), logloss(nn_pos)

(0.864963503649635,
 0.30275269850951353,
 0.9420821114369502,
 0.2918600827710242)

In [31]:
(0.864963503649635,
 0.30275269850951353,
 0.9420821114369502,
 0.2918600827710242)

(0.8613138686131386,
 0.3028432195846944,
 0.9420821114369502,
 0.2922430505408254)

In [32]:
train_df['gbc_preds'] = gbc.predict_proba(x_train).T[1]
test_df['gbc_preds'] = gbc.predict_proba(x_test).T[1]
train_df['rfc_preds'] = rfc.predict_proba(x_train).T[1]
test_df['rfc_preds'] = rfc.predict_proba(x_test).T[1]
train_df['etc_preds'] = etc.predict_proba(x_train).T[1]
test_df['etc_preds'] = etc.predict_proba(x_test).T[1]
train_df['nn_preds'] = model.predict_proba(x_train_norm)
test_df['nn_preds'] = model.predict_proba(x_test_norm)

In [33]:
train_df.head()

,d.gone.neg.l1,d.gone.neg.l2,d.neg.frac.l3,week,camp.length,base.poll,y2000,y2002,y2004,y2006,...,deminc,dem.contrib.l1,dem.contrib.l2,rep.contrib.l1,rep.contrib.l2,d.gone.neg,gbc_preds,rfc_preds,etc_preds,nn_preds
1,0,0,0.000000,-11,23,45.744681,0,1,0,0,...,1,38350.0,126536.00,132545.00,102559.36,0,0.235871,0.146,0.0,0.199028
2,0,0,0.000000,-10,23,45.744681,0,1,0,0,...,1,58035.0,38350.00,522327.00,132545.00,0,0.154327,0.084,0.0,0.283316
3,1,0,0.000000,-8,23,45.744681,0,1,0,0,...,1,167280.0,67068.67,177739.44,138533.00,1,0.921011,0.958,1.0,0.906002
4,1,1,0.000000,-7,23,45.744681,0,1,0,0,...,1,389947.0,167280.00,543465.00,177739.44,1,0.985290,0.998,1.0,0.974159
5,1,1,0.071429,-6,23,45.744681,0,1,0,0,...,1,202734.0,389947.00,2415314.50,543465.00,1,0.989649,1.000,1.0,0.994450


In [34]:
train_df.to_csv(f'{PATH}train_preds.csv')
test_df.to_csv(f'{PATH}test_preds.csv')